In [62]:
import pathlib as pl
import pysam
import collections as col
import numpy as np
import scipy
import scipy.stats as scistats
import pandas as pd
import re as re

print(pysam.__version__)
print(scipy.__version__)
print(np.__version__)
print(pd.__version__)


input_dir = pl.Path("/home/ebertp/work/data/raw_vcf")

vcfs = input_dir.glob("*.vcf.gz")

gt_map = lambda x: "." if x is None else x


class StatisticsSorter():
    
    loc_order = {
        "wg": 0,
        "genome": 0,
        "chrX": 23,
        "X": 23,
        "chrY": 24,
        "Y": 24,
        "chrM": 25,
        "chrMT": 25,
        "M": 25,
        "MT": 25
    }
    
    filter_order = {
        "PASS": 0,
        "RefCall": 1,
        "Refcall": 1
    }
    
    calltype_order = {
        "PRECISE": 0,
        "IMPRECISE": 1,
        "UNSPECIFIED": 2,
        "UNKNOWN": 3
    }
    
    vartype_order = {
        "SNV": 0,
        "INS": 1,
        "DEL": 2,
        "DUP": 3,
        "INV": 4,
        "BND": 5
    }
    
    attribute_order = {
        "GT:0/1": 0,
        "GT:1/1": 1,
        "GT:0/1": 2,
        "GT:1/0": 3,
        "GT:0/0": 4,
        "length": 5,
        "support": 6,
        "quality": 7
    }
    
    statistics_order = {
        "count": 0,
        "mean": 55
    }
    
    def __init__(self, locations, filters, calltypes, vartypes, attributes, statistics):
        self = self._init_location_order(locations)
        self = self._init_generic_order(filters, self.filter_order)
        self = self._init_generic_order(calltypes, self.calltype_order)
        self = self._init_generic_order(vartypes, self.vartype_order)
        self = self._init_generic_order(attributes, self.attribute_order)
        self = self._init_statistics_order(statistics)
        return None
    
    def _init_location_order(self, locations):
        
        order_max = max(self.loc_order.values())
        match_autosome = re.compile("^(chr)?[0-9]{1,2}$")
        for loc in sorted(locations):
            mobj = match_autosome.search(loc)
            if loc in self.loc_order:
                continue
            elif mobj is not None:
                order_num = int(loc.strip("chr"))
                self.loc_order[loc] = order_num
            else:
                order_max += 1
                self.loc_order[loc] = order_max
        return self
    
    def _init_statistics_order(self, statistics):
        
        order_max = max(self.statistics_order.values())
        for statistic in sorted(statistics):
            if statistic in self.statistics_order:
                continue
            elif "pct_" in statistic:
                pct_num = int(statistic.rsplit("_", 1)[-1])
                self.statistics_order[statistic] = pct_num
                order_max = max(pct_num, order_max)
            else:
                order_max += 1
                self.statistics_order[statistic] = order_max
        return self
    
    def _init_generic_order(self, values, value_map):
        
        order_max = max(value_map.values())
        for value in sorted(values):
            if value in value_map:
                continue
            else:
                order_max += 1
                value_map[value] = order_max
        return self   
                    
    def get_order_number(self, data_row):
        
        order_loc = self.loc_order[data_row["location"]]
        order_filter = self.filter_order[data_row["filter_status"]]
        order_calltype = self.calltype_order[data_row["call_type"]]
        order_vartype = self.vartype_order[data_row["variant_type"]]
        order_attribute = self.attribute_order[data_row["attribute"]]
        order_statistic = self.statistics_order[data_row["statistic"]]
        order_num = order_loc, \
            order_filter, \
            order_calltype, \
            order_vartype, \
            order_attribute, \
            order_statistic
        return order_num


count_stats = col.Counter()
agg_stats = col.defaultdict(list)
for vcf_file in vcfs:
    print(vcf_file.name)
    fixed_var_type = vcf_file.name.rsplit(".", 3)[-3]
    if fixed_var_type != "sv":
        continue
    with pysam.VariantFile(vcf_file) as vcf:
        for record in vcf.fetch():
            contig = record.contig
            filter_category = "|".join(record.filter.keys())
            if "PRECISE" in record.info.keys():
                calltype = "PRECISE"
            elif "IMPRECISE" in record.info.keys():
                calltype = "IMPRECISE"
            else:
                calltype = "UNSPECIFIED"
            if "SUPPORT" in record.info.keys():
                read_support = record.info["SUPPORT"]
            elif "RE" in record.info.keys():
                read_support = record.info["RE"]
            elif "RNAMES" in record.info.keys():
                read_support = len(record.info["RNAMES"])
            else:
                read_support = -1
            try:
                vartype = record.info["SVTYPE"]
            except KeyError:
                vartype = fixed_var_type
            try:
                varlen = abs(record.info["SVLEN"])
            except KeyError:
                ref_length = len(record.ref)
                assert ref_length > 0
                diff_lengths = set(abs(ref_length - len(alt)) for alt in record.alts)
                if len(diff_lengths) == 1:
                    varlen = max(1, diff_lengths.pop())
                else:
                    varlen = -1
            quality = record.qual
            vcf_samples = list(record.samples.keys())
            assert len(vcf_samples) == 1
            vcf_sample_name = vcf_samples[0]
            sample_info = dict(record.samples[vcf_sample_name].items())
            gt = sample_info["GT"]
            if read_support == -1:
                dp = sample_info["DP"]
                read_support = dp
            
            genotype = f"GT:{gt_map(gt[0])}/{gt_map(gt[1])}"
            count_stats[(contig, filter_category, calltype, vartype, genotype)] += 1
            count_stats[("genome", filter_category, calltype, vartype, genotype)] += 1
            agg_stats[(contig, filter_category, calltype, vartype, "length")].append(varlen)
            agg_stats[("genome", filter_category, calltype, vartype, "length")].append(varlen)
            agg_stats[(contig, filter_category, calltype, vartype, "quality")].append(quality)
            agg_stats[("genome", filter_category, calltype, vartype, "quality")].append(quality)
            agg_stats[(contig, filter_category, calltype, vartype, "support")].append(read_support)
            agg_stats[("genome", filter_category, calltype, vartype, "support")].append(read_support)
    
    summary = []
    for key, value in count_stats.items():
        row = list(key)
        row.extend(["count", value])
        summary.append(row)
    
    for key, values in agg_stats.items():
        row = list(key)
        data_array = np.array(values)
        pct_scores = scistats.scoreatpercentile(
            data_array,
            per=[1, 5, 25, 50, 75, 95, 99]
        )
        mean = data_array.mean()
        if row[-1] in ["length", "support"]:
            normalize = lambda x: int(round(x, 0))
        else:
            normalize = lambda x: float(round(x, 4))
        labels = [
            "mean", "pct_01", "pct_05",
            "Q1_pct_25", "median_pct_50",
            "Q3_pct_75", "pct_95", "pct_99"]
        values = [mean] + list(pct_scores)
        for l, v in zip(labels, values):
            norm_v = normalize(v)
            new_row = row + [l, norm_v]
            summary.append(new_row)
            
    df = pd.DataFrame.from_records(
        summary,
        columns=[
            "location", "filter_status", "call_type", "variant_type", "attribute", "statistic", "value"
        ]
    )
    sorter = StatisticsSorter(
        df["location"].unique(),
        df["filter_status"].unique(),
        df["call_type"].unique(),
        df["variant_type"].unique(),
        df["attribute"].unique(),
        df["statistic"].unique()
    )
    
    df["sort_order"] = df.apply(sorter.get_order_number, axis=1)
    df.sort_values("sort_order", ascending=True, inplace=True)
    #df.drop("sort_order", axis=1, inplace=True)
    print(df.head(30))
    raise

0.21.0
1.10.1
1.24.2
2.0.0
9-20b_hifi.mm2-deepvar.t2tv2.indel.vcf.gz
9-20b_hifi.mm2-sniffles.t2tv2.sv.vcf.gz


[E::idx_find_and_load] Could not retrieve index file for '/home/ebertp/work/data/raw_vcf/9-20b_hifi.mm2-sniffles.t2tv2.sv.vcf.gz'


    location filter_status call_type variant_type attribute      statistic   
3     genome          PASS   PRECISE          INS    GT:1/1          count  \
5     genome          PASS   PRECISE          INS    GT:0/1          count   
567   genome          PASS   PRECISE          INS    length         pct_01   
568   genome          PASS   PRECISE          INS    length         pct_05   
569   genome          PASS   PRECISE          INS    length      Q1_pct_25   
570   genome          PASS   PRECISE          INS    length  median_pct_50   
566   genome          PASS   PRECISE          INS    length           mean   
571   genome          PASS   PRECISE          INS    length      Q3_pct_75   
572   genome          PASS   PRECISE          INS    length         pct_95   
573   genome          PASS   PRECISE          INS    length         pct_99   
599   genome          PASS   PRECISE          INS   support         pct_01   
600   genome          PASS   PRECISE          INS   support     

RuntimeError: No active exception to reraise